# DATA ENCODING USANDO IBM QISKIT, BAIDU PADDLE e XANADU PENNYLANE

Para utilizarmos modelos de Machine Learning, precisamos primeiro representar os dados em inputs numéricos para o nosso sistema interpretá-los. Em Quantum Machine Learning (QML) também passamos por esse processo e aqui teremos um problema mais fundamental visto que estamos transformando, a príncipio, representações de objetos clássicos em quânticos. O nome mais popular na literatura deste processo é Data Encoding. 
Conforme [ref.IV], este processo é uma parte crítica dos algoritmos de QML e afeta diretamente seu poder computacional.
Neste artigo iremos estudar como esse processo é feito atualmente com base em nossas referências. Estudaremos os três métodos mais famosos de Data Encoding: Basis Encoding, Amplitude Encoding e Phase Encoding bem como exemplos de implementação. Para tanto, utilizaremos os frameworks fornecidos pela IBM Qiskit e Baidu Paddle Quantum. Iremos também, superficialmente, comparar os módulos.


## Bibliotecas

Aqui importamos nossas bibliotecas. Para instalar o paddle fazemos 'pip install paddle-quantum'.

In [1]:
import pennylane as qml
from pennylane import numpy as np

from pennylane.templates.embeddings import BasisEmbedding, AmplitudeEmbedding, AngleEmbedding

In [ ]:
from qiskit import Aer, QuantumCircuit, execute

In [ ]:
import math
import numpy as np

In [1]:
import paddle
from paddle_quantum.ansatz import Circuit
from paddle_quantum.gate import BasisEncoding, AmplitudeEncoding, AngleEncoding
import paddle_quantum as pq


C:\Users\luccas.marim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openfermion\hamiltonians\hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
C:\Users\luccas.marim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


### 1. Basis Encoding

#### Qiskit

In [ ]:
# Número de qubits = tamanho do nosso string binário
n=16
qc = QuantumCircuit(n)
x = '1011100010101110' #Bit clássico que queremos transformar em quântico
x=x[::-1]  #Inverte
for i in range(len(x)):
    if x[i] == '1':
        qc.x(i)
  
qc.draw()

In [ ]:
backend = Aer.get_backend('statevector_simulator')
result = execute(qc,backend).result().get_statevector()
result.draw('latex')

#### Baidu Paddle

In [ ]:
# Number of qubits = length of the classical binary string
n = 4
# Initialize the circuit
basis_enc = Circuit(n)
x = '1011'
# Add a Pauli X gate to the ith qubit if the ith classical bit is 1
for i in range(len(x)):
    if x[i] == '1':
        basis_enc.x(i)
  
print(basis_enc)

In [26]:
#Poderiamos usar:

#init_state = pq.state.zero_state(n)
#basis_quantum_state = basis_enc(init_state)
#print(basis_quantum_state)

#Mas o Paddle Quantum já tem uma função pronta pra isso:

built_in_basis_enc = BasisEncoding(num_qubits=n)
# Classical information x should be of type Tensor
x = paddle.to_tensor([1, 0, 1, 1])
built_in_basis_enc_state = built_in_basis_enc(feature=x)

print(built_in_basis_enc_state)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


#### PennyLane

In [ ]:
dev = qml.device('default.qubit', wires=6)

@qml.qnode(dev)
def circuit(data):
    for i in range(6):
        qml.Hadamard(i)
    for i in range(len(data)):
        BasisEmbedding(features=data[i], wires=range(6),do_queue=True)
    return  qml.state()

data=[[1,0,1,1,1,0],
      [1,0,0,0,0,1]]

circuit(data)

fig, aux = qml.draw_mpl(circuit)(data)
fig.show()

### 2. Amplitude Encoding

#### IBM Qiskit

In [ ]:
desired_state = [
    1 / math.sqrt(15.25) * 1.5,
    0,
    1 / math.sqrt(15.25) * -2,
    1 / math.sqrt(15.25) * 3]

qc = QuantumCircuit(2)
qc.initialize(desired_state, [0,1])

qc.decompose().decompose().decompose().decompose().decompose().draw()

#### Baidu Paddle

In [30]:
#Aqui usamos um dado clássico em forma de vetor. Precisamos normalizar este.
x = [-1,1,-2,6]
x = x / (np.linalg.norm(x))

# Numero de qubits
# N=3 n=log_2(N)
n = 2
built_in_amplitude_enc = AmplitudeEncoding(num_qubits=n)
# Altera a natureza da informação clásica em quântica (nesse caso, um tensor)
x = paddle.to_tensor([0.5, 0.5, 0.5])
state = built_in_amplitude_enc(x)

print(state)


[0.57735026+0.j 0.57735026+0.j 0.57735026+0.j 0.        +0.j]


#### PennyLane

In [ ]:
dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit(data):
    AmplitudeEmbedding(features=data, wires=range(2),normalize=True)
    return qml.state()

data = [6,-12.5,11.15,7]
circuit(data)

fig, aux = qml.draw_mpl(circuit)(data)
fig.show()

### 3. Angle Encoding

#### Qiskit

In [ ]:

qc = QuantumCircuit(3)
qc.ry(0, 0)
qc.ry(2*math.pi/4, 1)
qc.ry(2*math.pi/2, 2)
qc.draw()


#### Paddle

In [ ]:
# Números de qubits = Tamanho da string de informação classica
n = 3
# Inicializa o circuito
angle_enc = Circuit(n)
# x é a string clássica
x = paddle.to_tensor([np.pi, np.pi, np.pi], 'float64')
# Adiciona um layer para os gates RY
for i in range(len(x)):
    angle_enc.ry(qubits_idx=i, param=x[i])
        
print(angle_enc)

init_state = pq.state.zero_state(n)
angle_quan_state = angle_enc(init_state)

print([np.round(i, 2) for i in angle_quan_state.data.numpy()])


# Outra forma:
n = 3
built_in_angle_enc = AngleEncoding(num_qubits=n, encoding_gate="ry", feature=x)
x = paddle.to_tensor([np.pi, np.pi, np.pi], 'float64')
init_state = pq.state.zero_state(n)
state = built_in_angle_enc(state=init_state)

print([np.round(i, 2) for i in state.data.numpy()])

#### PennyLane

In [ ]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(feature_vector):
    qml.AngleEmbedding(features=feature_vector, wires=range(3), rotation='Z')
    qml.Hadamard(0)
    return qml.probs(wires=range(3))

X = [1,2,3]

print(qml.draw(circuit, expansion_strategy="device")(X))

##### Referências
1. https://www.youtube.com/watch?v=-sxlXNz7ZxU&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=10
2. https://quantumzeitgeist.com/quantum-encoding-an-overview/#:~:text=Basis%20encoding%20is%20primarily%20used%20when%20real%20numbers,into%20a%20quantum%20state%20in%20the%20computational%20basis.
3. https://qml.baidu.com/tutorials/machine-learning/encoding-classical-data-into-quantum-states.html
4. https://learn.qiskit.org/course/machine-learning/data-encoding